# Create train-val-test splits for DAN

## Pré-requis et documentation

Documentation:
- [Doc de l'API Arkindex](https://doc.arkindex.org/overview/api/)
- [Doc du client Python de bas-niveau](https://pypi.org/project/arkindex-client/)
- [Doc de Arkindex CLI (haut niveau)](https://cli.arkindex.org/)

A installer :

In [ ]:
!pip install arkindex-cli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Login

In [ ]:
!arkindex login --host arkindex.teklia.com --email solenn.tual@ign.fr

Loading API client… Done!
Password: 
Authentication successful
Slug to save profile as (default): prod
Set this profile as the default? (yes): yes


## Liste des pages annotées

In [ ]:
import pandas as pd
import os
import plotly.express as px
import json
from arkindex import ArkindexClient

In [ ]:
cli = ArkindexClient('cdf9b285c9da6e82cddbf18aa070dc471a13facb')

In [ ]:
#Retrieve corpus (project)
corpus = cli.request('ListCorpus')
for c in corpus:
    #print(c)
    if c["name"] == 'EPITA-IGN | Registre Napoléonien':
      corpus_id = c["id"]
      print(c)
      print(json.dumps(c, indent=2))

{'id': '59285827-bba3-42d4-99c7-e2436fe19f94', 'name': 'EPITA-IGN | Registre Napoléonien', 'description': 'Projet de Solenn', 'public': False, 'rights': ['read', 'write', 'admin'], 'types': [{'id': 'a2718e04-c140-47c2-886d-42cf5757bf04', 'slug': 'page', 'display_name': 'Page', 'folder': False, 'color': 'b828b3'}, {'id': 'fede5d95-e562-4f33-97e2-1fe18d20e2f1', 'slug': 'text_line', 'display_name': 'Text line', 'folder': False, 'color': 'f4f740'}, {'id': '8e98a93d-992c-4a32-a6da-2bd17abcb3a4', 'slug': 'text_zone', 'display_name': 'Text zone', 'folder': False, 'color': '0ac4be'}, {'id': '151d522f-09cf-4a6e-9874-d39f31b4512f', 'slug': 'double_page', 'display_name': 'Double page', 'folder': False, 'color': 'b8284c'}, {'id': '064c8649-08ce-4c59-b60b-ffa8a7802376', 'slug': 'folder', 'display_name': 'Folder', 'folder': True, 'color': 'f5fff5'}, {'id': '3da0de23-ea2d-48b7-9f6f-4b9120b7c911', 'slug': 'tab_header', 'display_name': 'Table Header', 'folder': False, 'color': '233990'}, {'id': 'e36317

In [ ]:
import pandas as pd
import re

def cleantable(elements):
  res = []

  for e in elements["results"]:
    rs1 = {}
    rs1["id"] = e["id"]#ID page elem dans ARkindex
    rs1["name"] = e["name"] #Page elem dans arkindex
    rs1["coords"] = e["zone"]["polygon"] #Coords
    rs1["image_id"] = e["zone"]["image"]["id"] #Id IIIF ARkindex
    rs1["image_name"] = e["zone"]["image"]["path"] #URL IIIF
    page_cote = e["zone"]["image"]["path"].replace('CADASTRE%2FETATS_DE_SECTION%2F%2F','')
    page_cote = page_cote.replace('.jpg','')
    page_cote = page_cote.replace('.tif','')
    rs1["commune"],rs1["dossier_cote"],rs1["image_cote"] = page_cote.split('%2F') #Commune
    rs1["image_index"] = rs1["image_cote"][rs1["image_cote"].rfind('_')+1:]
    rs1["image_index"] = re.sub(r'^[0]+','',rs1["image_index"]) #Numéro de la page
    rs1["image_url"] = e["zone"]["url"] #URL IIIF
    rs1["image_width"] = e["zone"]["image"]["width"]
    rs1["image_height"] = e["zone"]["image"]["height"]
    res.append(rs1)
  return res

In [ ]:
elements = cli.request('ListElementChildren',id="2c88832d-4e0d-4b37-b78f-ba235313bff6",type="page",class_id="fef896f5-f24f-4c11-b2e6-305756110fd4",recursive=True,folder=False,page_size=500,page=1)
dfelem1 = pd.DataFrame.from_dict(cleantable(elements))
display(dfelem1)

,id,name,coords,image_id,image_name,commune,dossier_cote,image_cote,image_index,image_url,image_width,image_height
0,0d98a470-fd41-4a59-a6de-696c8fc51b97,FRAD094_3P_000602_0096_P1,"[[0, 0], [0, 3264], [2272, 3264], [2272, 0], [...",e2221497-e2f2-4bad-b306-e37913bd3af8,CADASTRE%2FETATS_DE_SECTION%2F%2FSANTENY%2F3P_...,SANTENY,3P_000602,FRAD094_3P_000602_0096,96,https://iiif.geohistoricaldata.org/iiif/2/CADA...,4544,3264
1,11f414bb-4f41-4a46-812e-07fd5d29cb80,FRAD094_3P_000505_0009_P1,"[[0, 0], [0, 3264], [2272, 3264], [2272, 0], [...",341d5764-91f1-4392-9ebb-03d83cc48bff,CADASTRE%2FETATS_DE_SECTION%2F%2FRUNGIS%2F3P_0...,RUNGIS,3P_000505,FRAD094_3P_000505_0009,9,https://iiif.geohistoricaldata.org/iiif/2/CADA...,4544,3264
2,148e89fa-ba57-4821-8121-79834766f13f,FRAD094_3P_000505_0033_P1,"[[0, 0], [0, 3264], [2272, 3264], [2272, 0], [...",a058cab9-7597-4082-bf50-d73cc6d7cea6,CADASTRE%2FETATS_DE_SECTION%2F%2FRUNGIS%2F3P_0...,RUNGIS,3P_000505,FRAD094_3P_000505_0033,33,https://iiif.geohistoricaldata.org/iiif/2/CADA...,4544,3264
3,19b72710-fa26-4001-8484-5cf3d0a0b737,FRAD094_3P_000636_0062_P1,"[[0, 0], [0, 3264], [2272, 3264], [2272, 0], [...",9e139526-b7a6-4417-8a64-3511a44d36bb,CADASTRE%2FETATS_DE_SECTION%2F%2FVALENTON%2F3P...,VALENTON,3P_000636,FRAD094_3P_000636_0062,62,https://iiif.geohistoricaldata.org/iiif/2/CADA...,4544,3264
4,1a6d7d2e-2f65-4224-adb3-fc5ec7327bf7,FRAD094_3P_000048_01_0069,"[[0, 0], [0, 5658], [3506, 5658], [3506, 0], [...",94c0720e-2898-47ba-bc25-53d7d796723f,CADASTRE%2FETATS_DE_SECTION%2F%2FARCUEIL%2FFRA...,ARCUEIL,FRAD094_3P_000048_01,FRAD094_3P_000048_01_0069,69,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3506,5658
...,...,...,...,...,...,...,...,...,...,...,...,...
142,95553353-8ff7-436c-a926-4fa7f7bf5bd6,FRAD094_3P_000491_0070,"[[0, 0], [0, 4544], [3264, 4544], [3264, 0], [...",36859c48-286c-4921-be2b-4f27efbd67b2,CADASTRE%2FETATS_DE_SECTION%2F%2FLEPLESSIS%2F3...,LEPLESSIS,3P_000491,FRAD094_3P_000491_0070,70,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3264,4544
143,14d66385-bdaf-4b45-b592-de79bff93ba8,FRAD094_3P_000491_0081,"[[0, 0], [0, 4544], [3264, 4544], [3264, 0], [...",5ce6f7dd-db4b-4a1f-9a31-1c4a348cae18,CADASTRE%2FETATS_DE_SECTION%2F%2FLEPLESSIS%2F3...,LEPLESSIS,3P_000491,FRAD094_3P_000491_0081,81,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3264,4544
144,346be28d-98a8-4831-95f8-54d527c7c64c,FRAD094_3P_000781_0586,"[[0, 0], [0, 4544], [3264, 4544], [3264, 0], [...",2208b291-a620-45dc-96cf-09ea53478482,CADASTRE%2FETATS_DE_SECTION%2F%2FVITRY%2F3P_00...,VITRY,3P_000781,FRAD094_3P_000781_0586,586,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3264,4544
145,b950a73a-3b14-4d2a-a782-bd335ed68a79,FRAD094_3P_000491_0090,"[[0, 0], [0, 4544], [3264, 4544], [3264, 0], [...",c60e7ae3-e789-449b-868d-14b8ad276197,CADASTRE%2FETATS_DE_SECTION%2F%2FLEPLESSIS%2F3...,LEPLESSIS,3P_000491,FRAD094_3P_000491_0090,90,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3264,4544


## Add tags

In [ ]:
# Path
document_path = '/content/drive/MyDrive/THESE/Arkindex/details_archives_94_ETS.csv'

# Check if the document exists
if os.path.exists(document_path):
    print(f"{document_path} exists.")
else:
    print(f"{document_path} does not exist.")

/content/drive/MyDrive/THESE/Arkindex/details_archives_94_ETS.csv exists.


In [ ]:
meta = pd.read_csv(document_path)

In [ ]:
meta2 = meta[["Cote","Tag_groupe_doc","P_debut_sampling","P_fin_sampling","Num_couleur_gris"]]
meta2

,Cote,Tag_groupe_doc,P_debut_sampling,P_fin_sampling,Num_couleur_gris
0,FRAD094_3P_000017_01,AP_1822,1.0,7.0,COLOR
1,FRAD094_3P_000017_01,AV_1822_NB,8.0,93.0,COLOR
2,FRAD094_3P_000017_01,AV_1822_B,94.0,100.0,COLOR
3,FRAD094_3P_000026_01,AP_1822,1.0,73.0,COLOR
4,FRAD094_3P_000048_01,RECTIFICATION_1835,2.0,67.0,COLOR
...,...,...,...,...,...
160,NaN,NaN,NaN,NaN,NaN
161,NaN,NaN,NaN,NaN,NaN
162,NaN,NaN,NaN,NaN,NaN
163,NaN,NaN,NaN,NaN,NaN


In [ ]:
group = []
color = []
for i in range(len(dfelem1)):
  for j in range(len(meta2)):
    if (dfelem1["dossier_cote"].iloc[i] == meta2["Cote"].iloc[j] and float(dfelem1["image_index"].iloc[i]) >= meta2["P_debut_sampling"].iloc[j] and float(dfelem1["image_index"].iloc[i]) <= meta2["P_fin_sampling"].iloc[j]):
      group.append(meta2["Tag_groupe_doc"].iloc[j])
      color.append(meta2["Num_couleur_gris"].iloc[j])

df = dfelem1
df["tag"] = group
df["color"] = color
display(df)

,id,name,coords,image_id,image_name,commune,dossier_cote,image_cote,image_index,image_url,image_width,image_height,tag,color
0,0d98a470-fd41-4a59-a6de-696c8fc51b97,FRAD094_3P_000602_0096_P1,"[[0, 0], [0, 3264], [2272, 3264], [2272, 0], [...",e2221497-e2f2-4bad-b306-e37913bd3af8,CADASTRE%2FETATS_DE_SECTION%2F%2FSANTENY%2F3P_...,SANTENY,3P_000602,FRAD094_3P_000602_0096,96,https://iiif.geohistoricaldata.org/iiif/2/CADA...,4544,3264,AV_1822_NB,GREY
1,11f414bb-4f41-4a46-812e-07fd5d29cb80,FRAD094_3P_000505_0009_P1,"[[0, 0], [0, 3264], [2272, 3264], [2272, 0], [...",341d5764-91f1-4392-9ebb-03d83cc48bff,CADASTRE%2FETATS_DE_SECTION%2F%2FRUNGIS%2F3P_0...,RUNGIS,3P_000505,FRAD094_3P_000505_0009,9,https://iiif.geohistoricaldata.org/iiif/2/CADA...,4544,3264,AV_1822_NB,GREY
2,148e89fa-ba57-4821-8121-79834766f13f,FRAD094_3P_000505_0033_P1,"[[0, 0], [0, 3264], [2272, 3264], [2272, 0], [...",a058cab9-7597-4082-bf50-d73cc6d7cea6,CADASTRE%2FETATS_DE_SECTION%2F%2FRUNGIS%2F3P_0...,RUNGIS,3P_000505,FRAD094_3P_000505_0033,33,https://iiif.geohistoricaldata.org/iiif/2/CADA...,4544,3264,AV_1822_NB,GREY
3,19b72710-fa26-4001-8484-5cf3d0a0b737,FRAD094_3P_000636_0062_P1,"[[0, 0], [0, 3264], [2272, 3264], [2272, 0], [...",9e139526-b7a6-4417-8a64-3511a44d36bb,CADASTRE%2FETATS_DE_SECTION%2F%2FVALENTON%2F3P...,VALENTON,3P_000636,FRAD094_3P_000636_0062,62,https://iiif.geohistoricaldata.org/iiif/2/CADA...,4544,3264,AV_1822_NB,GREY
4,1a6d7d2e-2f65-4224-adb3-fc5ec7327bf7,FRAD094_3P_000048_01_0069,"[[0, 0], [0, 5658], [3506, 5658], [3506, 0], [...",94c0720e-2898-47ba-bc25-53d7d796723f,CADASTRE%2FETATS_DE_SECTION%2F%2FARCUEIL%2FFRA...,ARCUEIL,FRAD094_3P_000048_01,FRAD094_3P_000048_01_0069,69,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3506,5658,RECTIFICATION_1835,COLOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,95553353-8ff7-436c-a926-4fa7f7bf5bd6,FRAD094_3P_000491_0070,"[[0, 0], [0, 4544], [3264, 4544], [3264, 0], [...",36859c48-286c-4921-be2b-4f27efbd67b2,CADASTRE%2FETATS_DE_SECTION%2F%2FLEPLESSIS%2F3...,LEPLESSIS,3P_000491,FRAD094_3P_000491_0070,70,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3264,4544,AV_1822_NB,COLOR
143,14d66385-bdaf-4b45-b592-de79bff93ba8,FRAD094_3P_000491_0081,"[[0, 0], [0, 4544], [3264, 4544], [3264, 0], [...",5ce6f7dd-db4b-4a1f-9a31-1c4a348cae18,CADASTRE%2FETATS_DE_SECTION%2F%2FLEPLESSIS%2F3...,LEPLESSIS,3P_000491,FRAD094_3P_000491_0081,81,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3264,4544,AV_1822_NB,COLOR
144,346be28d-98a8-4831-95f8-54d527c7c64c,FRAD094_3P_000781_0586,"[[0, 0], [0, 4544], [3264, 4544], [3264, 0], [...",2208b291-a620-45dc-96cf-09ea53478482,CADASTRE%2FETATS_DE_SECTION%2F%2FVITRY%2F3P_00...,VITRY,3P_000781,FRAD094_3P_000781_0586,586,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3264,4544,AV_1822_NB,GREY
145,b950a73a-3b14-4d2a-a782-bd335ed68a79,FRAD094_3P_000491_0090,"[[0, 0], [0, 4544], [3264, 4544], [3264, 0], [...",c60e7ae3-e789-449b-868d-14b8ad276197,CADASTRE%2FETATS_DE_SECTION%2F%2FLEPLESSIS%2F3...,LEPLESSIS,3P_000491,FRAD094_3P_000491_0090,90,https://iiif.geohistoricaldata.org/iiif/2/CADA...,3264,4544,AV_1822_NB,COLOR


In [ ]:
df.to_csv(path_or_buf='/content/drive/MyDrive/THESE/ONTOLOGIE/pages_annotees.csv',index=False,encoding='utf8',sep='\t')

## Echantillonnage

Train/Val/test à créer


In [ ]:
df["stratification_id"] = df["tag"] + df["color"]

In [ ]:
# Use scikit-learn to split the dataset into a training set and a testing set
# We need stratification according to the column `Tag_groupe_doc` and a grouping according to the column `Commune`
from sklearn.model_selection import StratifiedGroupKFold
skgf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)  # split == 5 ⇔ 80% train, 20% test
for split_id, (train_index_, test_index) in enumerate(skgf.split(df, df['stratification_id'], groups=df['commune'])):
    if split_id > 0:
        break
    print("")
    print(f"Split {split_id}")
    train_ = df.iloc[train_index_]
    train_index, val_index = next(skgf.split(train_, train_['stratification_id'], groups=train_['commune']))
    train = train_.iloc[train_index]
    val = train_.iloc[val_index]

    test = df.iloc[test_index]
    print(f"Train size: {len(train)}")
    print(f"Val size: {len(val)}")
    print(f"Test size: {len(test)}")
    print("")
    print(f"Train Tag_groupe_doc: {train['stratification_id'].value_counts()}")
    print(f"Val Tag_groupe_doc: {val['stratification_id'].value_counts()}")
    print(f"Test Tag_groupe_doc: {test['stratification_id'].value_counts()}")
    print("")
    print(f"Train Commune: {train['commune'].value_counts()}")
    print(f"Val Commune: {val['commune'].value_counts()}")
    print(f"Test Commune: {test['commune'].value_counts()}")
    print("")
    print(train["image_cote"])


Split 0
Train size: 97
Val size: 22
Test size: 28

Train Tag_groupe_doc: AP_1822COLOR               36
RECTIFICATION_1835COLOR    28
AV_1822_NBGREY             13
AV_1822_NBCOLOR            11
RECTIFICATION_1835GREY      7
AP_1822GREY                 2
Name: stratification_id, dtype: int64
Val Tag_groupe_doc: AV_1822_NBGREY             5
AV_1822_NBCOLOR            5
RECTIFICATION_1835GREY     4
AP_1822GREY                3
AP_1822COLOR               3
RECTIFICATION_1835COLOR    2
Name: stratification_id, dtype: int64
Test Tag_groupe_doc: AV_1822_NBGREY             10
RECTIFICATION_1835COLOR     7
AP_1822COLOR                5
AV_1822_NBCOLOR             5
RECTIFICATION_1835GREY      1
Name: stratification_id, dtype: int64

Train Commune: IVRY               12
LEPERREUX          10
NOGENT             10
ALFORTVILLE         9
GENTILLY            7
MAISONSALFORT       6
ARCUEIL             6
BRY                 5
ORLY                5
VALENTON            4
CHARENTON           4
FRESNES  

In [ ]:
for sub in [train, val, test]:
  print(sub["color"].value_counts())

COLOR    75
GREY     22
Name: color, dtype: int64
GREY     12
COLOR    10
Name: color, dtype: int64
COLOR    17
GREY     11
Name: color, dtype: int64


In [ ]:
train["color"].value_counts()

## Check that dataset object exist

In [ ]:
#Retrieve corpus (project)
dataset = cli.request('ListCorpusDatasets',id='59285827-bba3-42d4-99c7-e2436fe19f94')
dataset

{'count': 1,
 'number': 1,
 'next': None,
 'previous': None,
 'results': [{'id': '032ae384-77dd-4ec8-868a-4348dd74897f',
   'name': 'ETS_s1',
   'description': "Dataset d'extraction des informations contenues dans les états de sections.",
   'sets': ['training', 'test', 'validation'],
   'set_elements': None,
   'state': 'open',
   'corpus_id': '59285827-bba3-42d4-99c7-e2436fe19f94',
   'creator': 'Solenn Tual',
   'task_id': None,
   'created': '2024-03-04T09:35:22.526581Z',
   'updated': '2024-03-04T09:35:22.526600Z'}]}

## Add images in dataset on Arkindex

In [ ]:
# Add images to train set
for i in range(len(train)):
  elementid = train['id'].iloc[i]
  t = cli.request('CreateDatasetElement',id='032ae384-77dd-4ec8-868a-4348dd74897f',body={"set":"training","element_id":elementid})

In [ ]:
# Add images to validation set
for i in range(len(val)):
  elementid = val['id'].iloc[i]
  t = cli.request('CreateDatasetElement',id='032ae384-77dd-4ec8-868a-4348dd74897f',body={"set":"validation","element_id":elementid})

In [ ]:
# Add images to validation set
for i in range(len(test)):
  elementid = test['id'].iloc[i]
  t = cli.request('CreateDatasetElement',id='032ae384-77dd-4ec8-868a-4348dd74897f',body={"set":"test","element_id":elementid})

After executing this script, go to Arkindex and download the SQLITE database containing the annotations